In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [2]:

url = 'https://www.truereligion.com/robots.txt'

In [3]:
headers_user = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

In [4]:
site = requests.get(url,allow_redirects=False,headers=headers_user)

In [5]:
robots = BeautifulSoup(site.content,'html.parser')

In [6]:
#basic info for what not to scrape
print(robots.prettify)

<bound method Tag.prettify of User-Agent: *
Disallow: /Account-Show/
Disallow: /Account-EditName/
Disallow: /Account-EditEmail/
Disallow: /Account-EditPassword/
Disallow: /Address-List/
Disallow: /PaymentInstruments-List/
Disallow: /Account-OrderStatus/
Disallow: /Order-History/
Disallow: /Account-PasswordResetDialog
Disallow: /Search-*?
Disallow: /*search*
Disallow: /Cart-Show/
Disallow: /COShipping-Start/
Disallow: /COSummary-Submit/
Disallow: /on/demandware.store/Sites-TrueReligion-Site
Disallow: /on/demandware.static/Sites-TrueReligion-Site/-/default/v
Disallow: /*?q=
Disallow: /*?j=
Disallow: /*srule
Disallow: /*&amp;dwvar
Disallow: /*prefn1
Disallow: /*prefv1

Sitemap: https://www.truereligion.com/sitemap_0.xml>


In [7]:
#creates a list containing the pages
def pagination(url):
  pagination =[]
  pagination.append(url)
  site = requests.get(url,headers=headers_user)
  soup = BeautifulSoup(site.content, 'html.parser')
  x = soup.find('a',class_="pagination_next_button")

  if x is not None:

    while x is not None:

      next = soup.find('a',class_="pagination_next_button").get('href')
      pagination.append(next)
      site = requests.get(next,headers=headers_user)
      soup = BeautifulSoup(site.content, 'html.parser')
      x = soup.find('a',class_="pagination_next_button")
  else:
    pagination.append(url)

  return pagination



In [8]:
def scrape(url):

  product_data = {
              'product_name':[],
              'brand_name':[],
              'original_price':[], 
              'strike_down_price':[], 
              'promo_price':[],
              'product_link':[],
              'first_image_link':[]
             }

  paginations = pagination(url)

  for i in paginations:
    site = requests.get(i,headers=headers_user)
    soup = BeautifulSoup(site.content, 'html.parser')
    product = soup.find_all('li',class_={'grid-tile odd-true in_stock','grid-tile odd-false in_stock'})

    for n in range(len(product)):
      ######
      #product_name
      if product[n].find('a',class_="name-link product-name") is not None:
        product_data['product_name'].append(product[n].find('a',class_="name-link product-name").text.strip())
      else:
        product_data['product_name'].append('None')

      #brand_name
      if product[n].find('a',class_="name-link product-name") is not None:
        x = product[n].find('a',class_="name-link product-name").text.strip().index(' ') #index of the first '(space)'
        brand = product[n].find('a',class_="name-link product-name").text.strip()
        brand = brand[:x]
        product_data['brand_name'].append(brand)
      else:
        product_data['brand_name'].append('None')

      #original price
      if product[n].find('span',class_="price-standard") is not None:
        product_data['original_price'].append(product[n].find('span',class_="price-standard").text.strip())
      else:
        product_data['original_price'].append('None')

      #strike_price
      if product[n].find('span',class_="price-sales strike-through") is not None:
        product_data['strike_down_price'].append(product[n].find('span',class_="price-sales strike-through").text.strip())
      else:
        product_data['strike_down_price'].append('None')
      
      #promo price
      if product[n].find('div',class_="price-promo") is not None:
        product_data['promo_price'].append(product[n].find('div',class_="price-promo").text.strip())
      else:
        product_data['promo_price'].append('None')

      #image
      if product[n].find('img',class_="firstimage") is not None:
        product_data['first_image_link'].append(product[n].find('img',class_="firstimage").get('src'))
      else:
        product_data['first_image_link'].append('None')

      #link
      if product[n].find('a',class_="name-link product-name") is not None:
        x = product[n].find('a',class_="name-link product-name").get('href')
        link = f'{i}{x}'
        product_data['product_link'].append(link)
      else:
        product_data['product_link'].append('None')
      ######

  return product_data

In [9]:
scrape_products = pd.DataFrame(scrape('https://www.truereligion.com/last-chance-womens-hoodies-and-sweats'))
scrape_products

,product_name,brand_name,original_price,strike_down_price,promo_price,product_link,first_image_link
0,COLLEGIATE SLIM JOGGER,COLLEGIATE,$119.00,$59.97,$29.98,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
1,VELOUR ZIP SWEATSHIRT,VELOUR,$119.00,$59.97,$29.98,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
2,BUDDHA CREWNECK SWEATSHIRT,BUDDHA,$119.00,$59.97,$29.98,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
3,COLOR BLOCK SWEATSHIRT,COLOR,$119.00,$99.97,$49.98,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
4,BUDDHA CREWNECK SWEATSHIRT,BUDDHA,$119.00,$59.97,$29.98,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
...,...,...,...,...,...,...,...
133,CAMO HIGH RISE LOGO JOGGER,CAMO,$119.00,None,None,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
134,WOMENS HOODED TRENCH,WOMENS,$249.00,None,None,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
135,MOCK CROP SWEATSHIRT,MOCK,$99.00,None,None,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...
136,UTILITY CREWNECK SWEATSHIRT,UTILITY,$119.00,$99.97,$49.98,https://www.truereligion.com/last-chance-women...,https://cdn-us-ec.yottaa.net/5d66e8af4f1bbf5e3...


In [10]:
#convert to excel file
scrape_products.to_excel('product_dataset_2.xlsx')